# Project - Search Engine

## Βήμα 1: Συλλογή Δεδομένων

### H συνάρτηση Crawl

In [95]:
import requests
from bs4 import BeautifulSoup

def crawl_wikipedia(url):
    data = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Εξαγωγή τίτλου και κειμένου
        title = soup.find("h1").text # βρίσκει h1 html tag (header1) και επιστρέφει το κείμενο του
        paragraphs = [p.text for p in soup.find_all("p")] # βρίσκει όλα τα p html tags (paragraph) και επιστρέφει το κείμενο τους
        content = "\n".join(paragraphs) # διαχωριστής των παραγράφων το σύμβολο " | "
        data.append({'title': title, 'content': content}) # προσθήκη τίτλου και περιεχομένου στη λίστα data
    else: 
        print(f"Error: {response.status_code}")
        print(f"URL: {url}")
    
    return data

### Χρηση της συνάρτησης Crawl

In [96]:
# Λίστα άρθρων για συλλογή 
articles = [
    "Science",
    "Technology",
    "Engineering",
    "Mathematics",
    "Artificial_intelligence",
    "Machine_learning",
    "Deep_learning",
    "Data_science",
    "Computer_science",
    "Programming_language",
    "Software_engineering",
    "Operating_system",
    "Computer_network",
    "Internet",
    "Art",
    "History",
    "Music",
    "Literature",
]
collected_data = []

for article in articles:
    url = f'https://en.wikipedia.org/wiki/{article}'
    collected_data.extend(crawl_wikipedia(url))

In [97]:
from pprint import pprint
def print_article(collected_data):
    print("Number of articles collected:", len(collected_data))
    print(f"1.Article Title: {collected_data[0]['title']}")
    print("  Content (first 100 words):")
    pprint(" ".join(collected_data[0]['content'].split()[:100]))
print_article(collected_data)

Number of articles collected: 18
1.Article Title: Science
  Content (first 100 words):
('Science is a systematic discipline that builds and organises knowledge in '
 'the form of testable hypotheses and predictions about the universe.[1][2] '
 'Modern science is typically divided into two or three major branches:[3] the '
 'natural sciences (e.g., physics, chemistry, and biology), which study the '
 'physical world; and the behavioural sciences (e.g., economics, psychology, '
 'and sociology), which study individuals and societies.[4][5] The formal '
 'sciences (e.g., logic, mathematics, and theoretical computer science), which '
 'study formal systems governed by axioms and rules,[6][7] are sometimes '
 'described as being sciences as well; however, they are often regarded as a '
 'separate field because they rely on deductive')


### Αποθήκευση σε JSON

In [98]:
import json
def save_json(data, filename):
    with open('Files/' + filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
# Αποθήκευση δεδομένων σε json αρχείο
save_json(collected_data, 'wiki_data.json')

### Αποθήκευση σε CSV 

In [99]:
import csv
def save_csv(data, filename):
    with open('Files/' + filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['title', 'content'])
        writer.writeheader()
        writer.writerows(data)

# Αποθήκευση δεδομένων σε csv αρχείο
save_csv(collected_data, 'wiki_data.csv')

## Βήμα 2: Προεπεξεργασία Κειμένου (Text Processing)

### Αφαιρεση πηγών (π.χ. [1])

In [100]:
from pprint import pprint
import re 
import pandas as pd

# Load collected data from CSV
collected_data_df = pd.read_csv('Files/wiki_data.csv')
collected_data = collected_data_df.to_dict(orient='records')
for d in collected_data:
    d['content'] = re.sub(r"\[\d+\]", "", d['content']) # regex για αντικατάσταση πηγών με κενό

print_article(collected_data)

Number of articles collected: 18
1.Article Title: Science
  Content (first 100 words):
('Science is a systematic discipline that builds and organises knowledge in '
 'the form of testable hypotheses and predictions about the universe. Modern '
 'science is typically divided into two or three major branches: the natural '
 'sciences (e.g., physics, chemistry, and biology), which study the physical '
 'world; and the behavioural sciences (e.g., economics, psychology, and '
 'sociology), which study individuals and societies. The formal sciences '
 '(e.g., logic, mathematics, and theoretical computer science), which study '
 'formal systems governed by axioms and rules, are sometimes described as '
 'being sciences as well; however, they are often regarded as a separate field '
 'because they rely on deductive')


### Αφαίρεση σημείων στίξης

In [101]:
import string

for punct in string.punctuation:
    for d in collected_data:
        d['content'] = d['content'].replace(punct, '') # αφαίρεση σημείων στίξης
print_article(collected_data)

Number of articles collected: 18
1.Article Title: Science
  Content (first 100 words):
('Science is a systematic discipline that builds and organises knowledge in '
 'the form of testable hypotheses and predictions about the universe Modern '
 'science is typically divided into two or three major branches the natural '
 'sciences eg physics chemistry and biology which study the physical world and '
 'the behavioural sciences eg economics psychology and sociology which study '
 'individuals and societies The formal sciences eg logic mathematics and '
 'theoretical computer science which study formal systems governed by axioms '
 'and rules are sometimes described as being sciences as well however they are '
 'often regarded as a separate field because they rely on deductive')


### Tokenization and Stemming

In [102]:
from nltk.tokenize import word_tokenize
import nltk
from pprint import pprint
porter = nltk.PorterStemmer()

tokens = []
stemmed_data = []

for d in collected_data: # For each article
    tokens = word_tokenize(d['content'])  # Tokenize content 
    stemmed_tokens = [porter.stem(t) for t in tokens]  # Stem each token
    stemmed_data.append({
        "title": d["title"],
        "stemmed_tokens": stemmed_tokens
    })

In [103]:
def print_tokens(data, tokens):
    print(f"1. Article Title: {data[0]['title']}")
    print("Tokens first 20 words: ")
    pprint(data[0][tokens][:20])
print_tokens(stemmed_data, "stemmed_tokens")

1. Article Title: Science
Tokens first 20 words: 
['scienc',
 'is',
 'a',
 'systemat',
 'disciplin',
 'that',
 'build',
 'and',
 'organis',
 'knowledg',
 'in',
 'the',
 'form',
 'of',
 'testabl',
 'hypothes',
 'and',
 'predict',
 'about',
 'the']


### Stop-word removal

In [104]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')

cleaned_data = []
cleaned_data_for_csv = []

for d in stemmed_data: # or lemmed_data (one of the two) - Επιλογή μεταξύ stemming και lemmatization??
    filtered_tokens = [t for t in d['stemmed_tokens'] if t.lower() not in stopwords]
    cleaned_data.append({
        "title": d["title"],
        "cleaned_tokens": filtered_tokens
    })
    cleaned_data_for_csv.append({
        "title": d["title"],
        "content": " ".join(filtered_tokens)
    })
    
print_tokens(cleaned_data, "cleaned_tokens")

1. Article Title: Science
Tokens first 20 words: 
['scienc',
 'systemat',
 'disciplin',
 'build',
 'organis',
 'knowledg',
 'form',
 'testabl',
 'hypothes',
 'predict',
 'univers',
 'modern',
 'scienc',
 'typic',
 'divid',
 'two',
 'three',
 'major',
 'branch',
 'natur']


### Αποθήκευση σε .json και .csv 

In [105]:
save_json(cleaned_data, 'wiki_data_cleaned.json')
save_csv(cleaned_data_for_csv, 'wiki_data_cleaned.csv')

## Βήμα 3: Ευρετήριο (Indexing)

In [106]:
import json
import pandas as pd

with open('Files/wiki_data_cleaned.json', 'r', encoding='utf-8') as file:
    wiki_data = json.load(file)

corpus = {}
for i, entry in enumerate(wiki_data):
    title = entry.get("title", f"sent{i}") 
    tokens = entry.get("cleaned_tokens", [])
    corpus[title] = {token: tokens.count(token) for token in tokens}

df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
print("First 15 columns:")
df.iloc[:, :15]

First 15 columns:


,art,describ,divers,rang,cultur,activ,center,around,work,util,creativ,imagin,talent,expect,evok
Art,282,6,2,3,27,11,2,2,61,2,17,3,1,1,1
Artificial intelligence,2,10,0,5,2,8,16,3,15,6,2,1,0,7,0
Computer network,0,3,3,3,0,3,0,1,1,1,0,0,0,0,0
Computer science,1,4,0,1,0,2,1,3,6,0,0,0,0,1,0
Data science,0,5,0,2,0,0,0,0,5,0,0,1,0,0,0
Deep learning,6,3,0,4,0,10,1,8,8,2,0,0,0,0,0
Engineering,7,4,0,3,1,2,0,4,13,3,1,1,0,4,0
History,5,3,3,2,26,9,0,5,9,0,1,0,0,1,0
Internet,2,7,2,9,1,14,3,4,16,0,2,0,0,2,0
Literature,9,3,2,0,15,0,0,4,43,0,4,0,0,0,0


### Αποθήκευση σε .json και .csv 

In [107]:
df.to_csv('Files/wiki_data_inverted_index.csv')
df.to_json('Files/wiki_data_inverted_index.json', indent=4)

## Βήμα 4: Μηχανή αναζήτησης (Search Engine)

### Επεξεργασία ερωτήματος (Query Processing)

In [108]:
from nltk.stem import PorterStemmer

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

def process_query(query, inverted_index):
    tokens = tokenize(query)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return evaluate_query(stemmed_tokens, inverted_index)

def tokenize(query):
    # Tokenize the query into terms and operators
    tokens = query.replace('AND', ' AND ').replace('OR', ' OR ').replace('NOT', ' NOT ').split()
    return tokens

def evaluate_query(tokens, inverted_index):
    # Initialize result set to all documents for NOT operations or None otherwise
    all_documents = get_all_documents(inverted_index)
    result = None
    operator = None

    for token in tokens:
        if token in ('AND', 'OR', 'NOT'):
            operator = token
        else:
            current_set = get_documents_for_term(token, inverted_index)

            if operator == 'AND':
                result = result & current_set if result is not None else current_set
            elif operator == 'OR':
                result = result | current_set if result is not None else current_set
            elif operator == 'NOT':
                # NOT modifies the current_set
                current_set = all_documents - current_set
                result = current_set if result is None else result & current_set
            else:
                result = current_set if result is None else result

    return result if result is not None else set()

def get_documents_for_term(term, inverted_index):
    # Retrieve the set of documents for a given term
    return {doc for doc, count in inverted_index.get(term, {}).items() if count > 0}

def get_all_documents(inverted_index):
    # Retrieve all unique documents in the inverted index
    return {doc for docs in inverted_index.values() for doc in docs}

### Κατάταξη αποτελεσμάτων (Ranking)

#### TF-IDF (Term Frequency-Inverse Document Frequency)

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import PorterStemmer

# Function to calculate TF-IDF
def tf_idf_retrieval(query, inverted_index):

    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(query.lower())  # Tokenize and lowercased
    query_tokens = [stemmer.stem(token) for token in tokens]

    # Build the corpus from the inverted index
    corpus = []
    document_names = list(next(iter(inverted_index.values())).keys())  # Extract document names

    # Create the document-term matrix
    for doc in document_names:
        doc_str = " ".join(
            [term for term, docs in inverted_index.items() if doc in docs and docs[doc] > 0]
        )
        corpus.append(doc_str)

    # Apply TF-IDF using TfidfVectorizer
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)

    # Query TF-IDF transformation
    query_vector = vectorizer.transform([" ".join(query_tokens)])

    # Calculate cosine similarity between query and documents
    cosine_similarities = cosine_similarity(query_vector, X).flatten()

    # Rank documents based on cosine similarity, excluding those with a score of 0
    ranked_docs = sorted(
        [(score, doc) for score, doc in zip(cosine_similarities, document_names) if score > 0],
        reverse=True
    )
    return ranked_docs

###  Vector Space Model (VSM)

In [110]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def vsm_retrieval(query, inverted_index):
    # Preprocess the inverted index into a format suitable for TF-IDF processing
    documents = {}
    for term, doc_scores in inverted_index.items():
        for doc, score in doc_scores.items():
            if doc not in documents:
                documents[doc] = ""
            documents[doc] += (term + " ") * score

    doc_names = list(documents.keys())  # Extract document names
    doc_texts = list(documents.values())  # Extract document texts

    # Preprocess the query using stemming
    stemmer = PorterStemmer()
    query_tokens = word_tokenize(query.lower())
    query_stemmed = " ".join([stemmer.stem(token) for token in query_tokens])

    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Fit the documents and transform both documents and query
    tfidf_matrix = vectorizer.fit_transform(doc_texts)
    query_vector = vectorizer.transform([query_stemmed])

    # Calculate cosine similarity
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Rank documents based on similarity scores, excluding those with a score of 0
    ranked_docs = sorted(
        [(doc_names[doc_idx], score) for doc_idx, score in enumerate(similarities) if score > 0],
        key=lambda x: x[1],
        reverse=True,
    )

    return ranked_docs

### Okapi BM25 (Probabilistic retrieval model)

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from nltk.stem import PorterStemmer
import numpy as np
import math

# Function to calculate BM25 scores
def bm25_score(inverted_index, query, k1=1.5, b=0.75):
    # Initialize stemmer and preprocess the query
    stemmer = PorterStemmer()
    query_terms = [stemmer.stem(term) for term in query.lower().split()]

    # Flatten inverted index to document-term matrix
    documents = {}
    for term, doc_scores in inverted_index.items():
        for doc, score in doc_scores.items():
            if doc not in documents:
                documents[doc] = {}
            documents[doc][term] = score

    # Document lengths and average document length
    doc_lengths = {doc: sum(terms.values()) for doc, terms in documents.items()}
    avg_doc_length = np.mean(list(doc_lengths.values()))

    # BM25 calculation
    N = len(documents)  # Total number of documents
    scores = {doc: 0 for doc in documents}

    for term in query_terms:
        if term in inverted_index:
            n_q = len(inverted_index[term])  # Number of documents containing the term
            idf = math.log((N - n_q + 0.5) / (n_q + 0.5) + 1)  # IDF for the term

            for doc, freq in inverted_index[term].items():
                term_freq = freq
                doc_length = doc_lengths[doc]

                # BM25 formula
                numerator = term_freq * (k1 + 1)
                denominator = term_freq + k1 * (1 - b + b * (doc_length / avg_doc_length))
                scores[doc] += idf * (numerator / denominator)

    # Sort documents by score, excluding those with a score of 0
    ranked_docs = sorted(
        [(doc, score) for doc, score in scores.items() if score > 0],
        key=lambda x: x[1],
        reverse=True
    )
    return ranked_docs

## Βήμα 5. Αξιολόγηση συστήματος:

### Αξιολόγηση συστήματος με Precision, Recall, F1, MAP

In [112]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_retrieval(results, docs):
    
    precision_scores = []
    recall_scores = []
    f1_scores = []
    map_scores = []
    
    for query, retrieved_docs in results.items():
        relevant_docs = set(docs[query])
        retrieved_docs = set(retrieved_docs)
        
        # Precision, Recall, F1-Score
        num_relevant_retrieved = len(relevant_docs & retrieved_docs)
        num_retrieved = len(retrieved_docs)
        num_relevant = len(relevant_docs)
        
        precision = num_relevant_retrieved / num_retrieved if num_retrieved > 0 else 0
        recall = num_relevant_retrieved / num_relevant if num_relevant > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        
        # MAP
        sorted_retrieved = list(retrieved_docs)
        ap = 0
        relevant_count = 0
        
        for i, doc in enumerate(sorted_retrieved, 1):
            if doc in relevant_docs:
                relevant_count += 1
                ap += relevant_count / i
        
        ap /= len(relevant_docs) if len(relevant_docs) > 0 else 1
        map_scores.append(ap)
    
    # Compute averages
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_recall = sum(recall_scores) / len(recall_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    mean_ap = sum(map_scores) / len(map_scores)

    evaluation_results = {
        "Precision": avg_precision,
        "Recall": avg_recall,
        "F1-Score": avg_f1,
        "MAP": mean_ap
    }
    
    return evaluation_results

## Διεπαφή Χρήστη (User Interface)

### Διάβασμα άρθρων και ευρετηρίου

In [113]:
import json

# Load articles
with open("Files/wiki_data_cleaned.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

# Load inverted index
with open("Files/wiki_data_inverted_index.json", "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

### Διεπαφή χρήστη

In [121]:
tf_results = {}
tf_queries = {}
vsm_results = {}
vsm_queries = {}
bm25_results = {}
bm25_queries = {}

def user_interface():
    while True:
        questions = [
            "---------------Search Engine Menu---------------",
            "1. Boolean search",
            "2. TF-IDF Ranking",
            "3. Vector Space Model Ranking",
            "4. BM25 Ranking",
            "5. Evaluate System",
            "6. Exit",
        ]
        print("\n".join(questions))
        choice = input("Choose:")

        article_titles = [article["title"] for article in articles]

        if choice == "1":
            query = input("Input Boolean query (e.g. term1 AND term2): ")
            results = process_query(query, inverted_index)
            if not results:
                print("No results found.")
                break

            print("Results:")
            for res in results:
                print(res)
        elif choice == "2":
            query = input("Input query:")
            print("---------------Results TF-IDF---------------")
            print("Query given:", query)
            ranked_docs = tf_idf_retrieval(query, inverted_index)
            if not ranked_docs:
                print("No results found.")
                break

            for score, doc in ranked_docs:
                    print(f"Document: {doc}, Score: {score:.4f}")
            tf_results[query] = [doc for _, doc in ranked_docs]
            tf_queries[query] = article_titles
        elif choice == "3":
            query = input("Input query:")
            print("---------------Results Vector Space Model---------------")
            print("Query given:", query)
            ranked_docs = vsm_retrieval(query, inverted_index)
            if not ranked_docs:
                print("No results found.")
                break

            for doc, score in ranked_docs:
                print(f"Document: {doc}, Score: {score:.4f}")
            vsm_results[query] = [doc for doc, _ in ranked_docs]
            vsm_queries[query] = article_titles
        elif choice == "4":
            query = input("Input query:")
            print("---------------Results BM25---------------")
            print("Query given:", query)
            ranked_docs = bm25_score(inverted_index, query)
            if not ranked_docs:
                print("No results found.")
                break

            for doc, score in ranked_docs:
                print(f"Document: {doc}, Score: {score:.4f}")
            bm25_results[query] = [doc for doc, _ in ranked_docs]
            bm25_queries[query] = article_titles
        elif choice == "5":

            print("Αξιολόγηση TF-IDF:")
            if tf_results:
                tf_idf_results = evaluate_retrieval(tf_results, tf_queries)
                for metric, value in tf_idf_results.items():
                    print(f"{metric}: {value:.4f}")
            else:   
                print("Δεν υπάρχουν αποτελέσματα για αξιολόγηση.")

            print("Αξιολόγηση VSM:")
            if vsm_results:
                results = evaluate_retrieval(vsm_results, vsm_queries)
                for metric, value in results.items():
                    print(f"{metric}: {value:.4f}")
            else:
                print("Δεν υπάρχουν αποτελέσματα για αξιολόγηση.")

            print("Αξιολόγηση BM25:")
            if bm25_results:
                results = evaluate_retrieval(bm25_results, bm25_queries)
                for metric, value in results.items():
                    print(f"{metric}: {value:.4f}")
            else:
                print("Δεν υπάρχουν αποτελέσματα για αξιολόγηση.")

        elif choice == "6" or choice == "": 
            break


In [120]:
user_interface()

---------------Search Engine Menu---------------
1. Boolean search
2. TF-IDF Ranking
3. Vector Space Model Ranking
4. BM25 Ranking
5. Evaluate System
6. Exit
---------------Results TF-IDF---------------
Query given: art animation
Document: Computer science, Score: 0.0297
Document: History, Score: 0.0254
Document: Engineering, Score: 0.0226
Document: Literature, Score: 0.0213
Document: Mathematics, Score: 0.0212
Document: Technology, Score: 0.0208
Document: Machine learning, Score: 0.0207
Document: Science, Score: 0.0204
Document: Deep learning, Score: 0.0188
Document: Art, Score: 0.0179
Document: Internet, Score: 0.0169
Document: Music, Score: 0.0163
Document: Artificial intelligence, Score: 0.0154
Document: Software engineering, Score: 0.0139
---------------Search Engine Menu---------------
1. Boolean search
2. TF-IDF Ranking
3. Vector Space Model Ranking
4. BM25 Ranking
5. Evaluate System
6. Exit
---------------Results TF-IDF---------------
Query given: history lesson
Document: Histo